In [ ]:
import BubbleCount.image_preprocess as image_preprocess
import BubbleCount.csv_helpers as csv_helpers

import csv
%matplotlib inline

ImportError: cannot import name '_imaging' from 'PIL' (c:\Users\libr3\Desktop\Summer_2025\fellowship 2025\test\BubbleCount-with-FamNet\.venv\Lib\site-packages\PIL\__init__.py)

In [ ]:
from BubbleCount.counting_model import CountingPipe
model = CountingPipe()  # Only really need the count_hybrid() function in this notebook

In [ ]:
args = {
    "sample_path": "./Exemplars/",
    "target_path": "./2025/Targets", # cropped images
    "result_path": "./2025/Outputs/out.csv", # csv containing counts
    "output_dir": "./2025/Outputs/", # output images
    "model_path": "./data/pretrainedModels/FamNet_Save1.pth",
    "raw_img_dir": "./2025/Images/SEN10_1.8_6_300-400" # images to be cropped
}

In [ ]:
# Crop images to AOI
image_preprocess.crop_to_interest(image_dir=args["raw_img_dir"],region=[65, 770, 1090, 970],output_dir=args["target_path"])

In [ ]:
def simplify_target_name(target):
    temp = target.split('_')[1:-1]
    temp = '_'.join(temp)
    serial_number = temp.split('.')[-1]
    batch_name = '.'.join(temp.split('.')[0:-2])

    return f"{batch_name}_{serial_number}"

In [ ]:
result_to_csv = []

# Load exemplars and targets
Exemplars = image_preprocess.load_exemplars_from_directory(args["sample_path"], reverse_bbox=False)
Targets = image_preprocess.load_target_images_from_directory(args["target_path"])

sample_image = Exemplars[1]

csv_helpers.backup_and_clear_csv(args["result_path"])

with open(args["result_path"], 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Exemplar", "num_exemps", "rev_bbox", "Target", "Count"])

    total = len(Targets)
    count = 0
    
    for target_image in Targets:
        num_exemps = 4
        # Creating the hybrid
        hybrid, hybrid_boxes = image_preprocess.insert_cropped(sample_image['image'], target_image['image'], sample_image['box'], num_exemps)
        target_name = f"{simplify_target_name(target_image['file_name'])}_{num_exemps}exemps"

        # counting
        hybrid_count = model.count_hybrid_and_visualize_separate(hybrid, hybrid_boxes, sample_image["file_name"], target_name,output_directory=args["output_dir"])

        result_to_csv.append([sample_image['file_name'], num_exemps, False, target_name, hybrid_count])

        # append to csv
        writer.writerows(result_to_csv)

        count = count + 1
        print(f"Visualized: {count}/{total}")

print(f"The counts are saved to {args['result_path']}.")

In [ ]:
import cv2
import os

# Generate videos from images
dir = "Overlaid" # "Density" or "Graphs" or "Overlaid"
fps = 10

input_folder = f'{args["output_dir"]}{dir}/'

print(input_folder)

# Get the list of image files in the input folder
image_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.jpg') or f.endswith('.png')])

# Read the first image to get its dimensions
first_image = cv2.imread(os.path.join(input_folder, image_files[0]))
height, width, _ = first_image.shape

# Create a VideoWriter object to save the video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Specify the codec for the output video file
video = cv2.VideoWriter(f"{input_folder}video.mp4", fourcc, fps, (width, height))

# Iterate over each image and write it to the video
for image_file in image_files:
    print(image_file)
    image_path = os.path.join(input_folder, image_file)
    frame = cv2.imread(image_path)
    video.write(frame)

# Release the video writer and close the video file
video.release()
cv2.destroyAllWindows()
